In [2]:
import logging
import sys
import os

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '4'
os.environ['NUMEXPR_NUM_THREADS'] = '2'
import numexpr as ne

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
import openai
openai.api_key = "sk-xxx"

In [16]:
from llama_index import SimpleDirectoryReader
andrew_gina_docs = SimpleDirectoryReader(input_files=["./assets/AndrewHuberman/sleep/115_Dr_Gina_Poe_Use_Sleep_to_Enhance_Learning_Memory_&_Emotional_State_Huberman_Lab_Podcast.txt"]).load_data()

In [17]:
print(len(andrew_gina_docs))

1


In [21]:
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor
)
from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=20)

metadata_extractor = MetadataExtractor(
    extractors=[
        TitleExtractor(nodes=5),
        QuestionsAnsweredExtractor(questions=3),
    ],
)

node_parser = SimpleNodeParser(
    text_splitter = text_splitter,
    metadata_extractor=metadata_extractor
)

In [23]:
andrew_gina_nodes = node_parser.get_nodes_from_documents(andrew_gina_docs)

Extracting questions: 100%|██████████| 57/57 [03:12<00:00,  3.38s/it]


In [24]:
print(len(andrew_gina_nodes))

57


In [25]:
andrew_gina_nodes[1].metadata

{'document_title': 'The Comprehensive Guide to Sleep: Optimizing Health, Performance, and Well-being',
 'questions_this_excerpt_can_answer': '1. How can sleep be used to optimize learning and forgetting?\n2. What critical information about neuroscience, physiology, and hormone systems can Dr. Gina Poe share that strongly inform mental health, physical health, and performance?\n3. What actionable steps can be taken to improve mental health, physical health, and performance through sleep optimization?'}

In [26]:
import random
sample_gina_nodes = random.sample(andrew_gina_nodes, 5)
for node in sample_gina_nodes:
    print(node.metadata)

{'document_title': 'The Comprehensive Guide to Sleep: Optimizing Health, Performance, and Well-being', 'questions_this_excerpt_can_answer': '1. How do antidepressants affect REM sleep and why are they counter-indicated for individuals with PTSD?\n2. What is the role of serotonin in REM sleep and how does its regulation impact cognition and mental health?\n3. How do different types of antidepressants, such as noradrenergic and serotonergic reuptake inhibitors, affect sleep patterns and emotional processing?'}
{'document_title': 'The Comprehensive Guide to Sleep: Optimizing Health, Performance, and Well-being', 'questions_this_excerpt_can_answer': "1. How does the locus coeruleus in the brain contribute to ignoring pain during stressful situations?\n2. What role does the dorsal striatum play in procedural learning and motor learning when the hippocampus is compromised?\n3. Can manipulating the body's adrenal glands have an impact on brain function and overall well-being?"}
{'document_tit

In [27]:
from llama_index import ServiceContext, VectorStoreIndex

service_context = ServiceContext.from_defaults(node_parser=node_parser)
index = VectorStoreIndex.from_documents(andrew_gina_docs, service_context=service_context)

Extracting questions: 100%|██████████| 57/57 [03:16<00:00,  3.44s/it]


In [31]:
response = index.as_query_engine().query("How does consistent bedtime contribute to good neurological health as we age?")

In [32]:
from llama_index.response.pprint_utils import pprint_response
pprint_response(response, show_source=True)

Final Response: Consistent bedtime contributes to good neurological
health as we age by serving as one of the best markers of good
neurological health. It helps maintain the proper functioning of our
brains as we get older.
______________________________________________________________________
Source Node 1/2
Node ID: 4ce03611-578e-482d-9714-c945d76be165
Similarity: 0.9152142163390966
Text: are probably realizing, also highly actionable. So what this
means is that we should have fairly consistent bedtimes in addition to
fairly consistent, wait times. Is that right exactly and in fact one
of the Best markers of good neurological health when we get older, is
consistent, bedtimes, wow, okay, um. I don't want to backtrack, but I
did wr...
______________________________________________________________________
Source Node 2/2
Node ID: 2a6027eb-fef3-4db3-b58b-b803f4e76dd7
Similarity: 0.8429651446701504
Text: Welcome to the Huberman Lab podcast, where we discuss science
and science-based tools

In [36]:
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='4ce03611-578e-482d-9714-c945d76be165', embedding=None, metadata={'document_title': 'The Comprehensive Guide to Sleep: Understanding its Importance for Health, Well-being, Mental Health, Physical Health, Performance, and Finding the Perfect Mattress', 'questions_this_excerpt_can_answer': '1. How does consistent bedtime contribute to good neurological health as we age?\n2. What are the potential underlying issues for individuals who sleep nine hours or more?\n3. How does sleep aid in brain and body development, particularly in babies?'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['questions_this_excerpt_can_answer'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='dfed5f6f-99cb-4b09-b5af-ff4d4f2a0b09', node_type=None, metadata={}, hash='fbcacda02d943cacab9cdd5d314b1865ca59a36997b288e52d3c59fd535a314b'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='52e4c512-32b0-4f77-9958-21abf9b18727', node_type=Non